In [10]:
import tensorflow as tf
from influxdb.commands import InfluxDBQueries, DataProcessor
import pandas as pd
tf.config.set_visible_devices([], 'GPU')

In [11]:
model_5m = tf.keras.models.load_model('./models/best_model_cnn_lstm_5m.keras')

In [12]:
model_1h = tf.keras.models.load_model('./models/best_model_cnn_lstm_1h.keras')

In [13]:
model_1d = tf.keras.models.load_model('./models/best_model_cnn_lstm_1d.keras')

In [15]:
df_train = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2021-01-01T01:00:00Z",
                                                stop="2025-01-01T01:00:00Z")
df_val = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-01-01T01:00:00Z",
                                              stop="2025-03-01T01:00:00Z")
df_test = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-03-01T01:00:00Z",
                                               stop="2025-04-20T01:00:00Z")
df_combined = pd.concat([df_train, df_test, df_val])

In [16]:
df_combined, scaler, columns, sclaer_y = DataProcessor.add_features(df_combined, classification=True, group_time='5m')

In [18]:
df_combined.dropna(inplace=True)

In [19]:
df_test = df_combined.loc[
    df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]
df_test = df_test.dropna()

In [22]:
df_test

,low,tb_base,open,volume,high,close,quote_volume,trades,tb_quote,return_1,...,range_cur,range_pct,price,beta,market_return,expected_return,actual_return,alpha,risk_free_rate,target
time,,,,,,,,,,,,,,,,,,,,,
2025-03-01 01:05:00+00:00,1.716830,-0.478987,1.712844,-0.489573,1.714265,1.718264,-0.397230,0.228107,-0.371271,-52.865803,...,0.161258,-0.351311,1.718264,0.762095,-0.132320,-0.151998,-0.136889,-0.008182,0.0,0
2025-03-01 01:10:00+00:00,1.718614,-0.539997,1.718276,-0.523068,1.718115,1.716953,-0.504100,0.018972,-0.563868,-0.167464,...,0.541392,-0.130281,1.716953,0.761009,-0.092581,-0.104894,-0.092221,-0.000721,0.0,1
2025-03-01 01:15:00+00:00,1.721229,-0.517058,1.716964,-0.528280,1.716698,1.719069,-0.520692,-0.177345,-0.491306,0.264669,...,-0.197965,-0.559995,1.719069,0.760991,-0.089759,-0.101550,-0.088351,0.001341,0.0,1
2025-03-01 01:20:00+00:00,1.723347,-0.545561,1.719081,-0.527215,1.718009,1.722101,-0.517156,-0.297629,-0.581345,0.379832,...,-0.345505,-0.645913,1.722101,0.762494,0.057647,0.073157,-0.024423,-0.194090,0.0,0
2025-03-01 01:25:00+00:00,1.723538,-0.526204,1.722113,-0.538063,1.720424,1.721672,-0.551863,-0.179803,-0.520089,-0.056235,...,0.062926,-0.408918,1.721672,0.771115,0.202425,0.245320,0.081547,-0.292444,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-20 00:40:00+00:00,1.775833,-0.571009,1.772827,-0.568293,1.769525,1.771854,-0.646941,-0.482620,-0.660590,-0.121621,...,-0.504413,-0.741145,1.771854,-0.683901,0.384997,0.303134,0.229478,-0.079121,0.0,0
2025-04-20 00:45:00+00:00,1.773805,-0.578795,1.771865,-0.584420,1.767661,1.769771,-0.699305,-0.495789,-0.685600,-0.261131,...,-0.474955,-0.724142,1.769771,-0.688625,0.356468,0.278751,0.206940,-0.081702,0.0,0
2025-04-20 00:50:00+00:00,1.772681,-0.562338,1.769783,-0.567500,1.766878,1.769605,-0.644474,-0.485166,-0.632882,-0.022772,...,-0.412802,-0.688522,1.769605,-0.689404,0.359193,0.280950,0.213108,-0.072403,0.0,1


In [20]:
X_test, y_test = DataProcessor.make_sequences(df_test, columns_to_select=columns, window_size=288, forecast_horizon=1, classification=True)

In [23]:
model_5m.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 288, 64)        │         4,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 288, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 288, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 288, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 288, 128)       │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 288, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 288, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 80)             │        66,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 80)             │           320 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 512,373 (1.95 MB)

 Trainable params: 170,609 (666.44 KB)

 Non-trainable params: 544 (2.12 KB)

 Optimizer params: 341,220 (1.30 MB)